In [59]:

# Import the required libraries and dependencies
import os
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import sqlalchemy


In [60]:

def retrieve_data(tickers, start_date, end_date):
    """Retrieve historical data for a list of tickers within a specified time period.
    
    Args: 
        tickers (list) = List of tickers to retrieve data for.
        start_date(Timestamp) = The start date of the historical data.
        end_date(Timestamp) = The end date of the historical data.
    
    Returns:
        A DataFrame containing the historical data for the specified tickers.
        
    """
    
    # Load the environment variables from the .env file
    #by calling the load_dotenv function
    load_dotenv()

    # Set the variables for the Alpaca API and secret keys
    alpaca_api_key= os.getenv("ALPACA_API_KEY")
    alpaca_secret_key= os.getenv("ALPACA_SECRET_KEY")

    # Create the Alpaca tradeapi.REST object
    alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")
    
    # Set timeframe to 1Day
    timeframe = "1Day"
    
    # Use the Alpaca get_bars function to get historical data of the assets
    portfolio_data_df = alpaca.get_bars(
        tickers,
        timeframe,
        start = start_date,
        end = end_date
     ).df
    
    return portfolio_data_df
    

In [67]:

def store_data_sql(data):
    """Stores data into separate tables in an SQL database based on unique symbols.
    
    Args:
        data (DataFrame): The DataFrame containing the data to be stored in the SQL database.
    
    """
    
    # Create the connection string for your SQLite database
    database_connection_string = 'sqlite:///'

    # Pass the connection string to the SQLAlchemy create_engine function
    engine = sqlalchemy.create_engine(database_connection_string)
    
    # Iterate over the unique symbols and save each symbol's data into a separate table
    for symbol in data['symbol'].unique():
        symbol_data = data[data['symbol'] == symbol]
        symbol_data.to_sql(name=symbol, con=engine, if_exists='replace')




In [68]:
def run():
    """The main function for running the script."""
    
    #---------------- this will be set by the user ------------
    # Define tickers for both the bond and stock portion of the portfolio
    tickers = ["SPY", "AGG"]

    # Define start and end dates
    start_date = pd.to_datetime("2023-05-9").strftime("%Y-%m-%d")
    end_date = pd.to_datetime("2023-06-9").strftime("%Y-%m-%d")
    #------------------------------------------------------
    
    # Retrieve the historical data
    data_df = retrieve_data(tickers, start_date, end_date)
    
    # Store data in SQL database
    store_data_sql(data_df)
   
    

In [69]:
run()
